# Import the libraries

In [1]:
# install the opencage to get the geocoordinates for each country
#%pip install opencage
# install pycountry to get the ISO country code
#%pip install pycountry


In [1]:
# Dependencies
import pandas as pd
from functools import reduce
from pathlib import Path
from opencage.geocoder import OpenCageGeocode
# For interactive visualisations
import plotly.express as px
from ipywidgets import interact, IntSlider
import pycountry # to convert numeric code to ISO country code used by plotly
# for using geopandas and matplotlib
import geopandas as gpd
import matplotlib.pyplot as plt


# Define paths and read files

In [2]:
# Store filepath in a variable
file_one = Path("cleaneddata/morticd10_part4_cleaned_df.csv")
file_two = Path("cleaneddata/morticd10_part5_cleaned_df.csv")
file_3 = Path("data/country_codes.csv")

In [3]:
# Read our data file with the Pandas library
# Not every CSV requires an encoding, but be aware this can come up
morticd10_part4_df = pd.read_csv(file_one, encoding="ISO-8859-1")
morticd10_part5_df = pd.read_csv(file_two, encoding="ISO-8859-1")
country_code_df = pd.read_csv(file_3, encoding="ISO-8859-1")


# Understanding/analysing the data files

In [7]:
country_code_df

,ï»¿country,name
0,1010,Algeria
1,1020,Angola
2,1025,Benin
3,1030,Botswana
4,1035,Burkina Faso
...,...,...
222,5197,Samoa
223,5198,Solomon Islands
224,5200,Tonga
225,5205,Tuvalu


In [6]:
morticd10_part4_df

,Country,Year,List,Cause,Sex,Deaths1,IM_Deaths1,IM_Deaths2,IM_Deaths3,IM_Deaths4,mort_1_4,mort_5_24,mort_25_44,mort_45_64,mort_65_84,mort_85_95+,Deaths26
0,1400,2013,101,1000,1,383,5.0,1.0,2.0,4.0,0.0,6.0,44.0,103.0,218.0,0.0,0.0
1,1400,2013,101,1000,2,334,2.0,7.0,3.0,5.0,2.0,3.0,20.0,60.0,232.0,0.0,0.0
2,1400,2013,101,1001,1,28,0.0,0.0,0.0,0.0,0.0,0.0,6.0,11.0,11.0,0.0,0.0
3,1400,2013,101,1001,2,30,0.0,2.0,2.0,1.0,0.0,0.0,0.0,6.0,19.0,0.0,0.0
4,1400,2013,101,1002,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880537,5020,2016,104,Y870,2,2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
880538,5020,2016,104,Y883,1,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,1.0,0.0
880539,5020,2016,104,Y883,2,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,3.0,0.0
880540,5020,2016,104,Y899,1,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,0.0


In [8]:
morticd10_part5_df

,Country_code,Year,List,Cause,Sex,Total_deaths,Mort_day0,Mort_day1_6,Mort_day7_27,Mort_day28_365,mort_1_4,mort_5_24,mort_25_44,mort_45_64,mort_65_84,mort_85_95+,Deaths26
0,4303,2017,101,1000,1,281784,1608,0,0,0,357,2593,26494,86787,134693,29184,68
1,4303,2017,101,1000,2,292339,1178,0,0,0,241,998,9009,37295,155328,88271,19
2,4303,2017,101,1001,1,6198,41,0,0,0,29,65,3019,2670,358,15,1
3,4303,2017,101,1001,2,2516,29,0,0,0,19,51,1317,808,263,29,0
4,4303,2017,101,1002,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023255,5020,2022,104,Y871,1,1,0,0,0,0,0,0,1,0,0,0,0
1023256,5020,2022,104,Y883,1,4,0,0,0,0,0,0,2,0,1,1,0
1023257,5020,2022,104,Y883,2,3,0,0,0,0,0,0,0,1,2,0,0
1023258,5020,2022,104,Y890,2,1,0,0,0,0,0,1,0,0,0,0,0


In [9]:
part5_cols = morticd10_part5_df.columns
part5_cols

Index(['Country_code', 'Year', 'List', 'Cause', 'Sex', 'Total_deaths',
       'Mort_day0', 'Mort_day1_6', 'Mort_day7_27', 'Mort_day28_365',
       'mort_1_4', 'mort_5_24', 'mort_25_44', 'mort_45_64', 'mort_65_84',
       'mort_85_95+', 'Deaths26'],
      dtype='object')

In [10]:
part4_cols = morticd10_part4_df.columns
part4_cols

Index(['Country', 'Year', 'List', 'Cause', 'Sex', 'Deaths1', 'IM_Deaths1',
       'IM_Deaths2', 'IM_Deaths3', 'IM_Deaths4', 'mort_1_4', 'mort_5_24',
       'mort_25_44', 'mort_45_64', 'mort_65_84', 'mort_85_95+', 'Deaths26'],
      dtype='object')

In [11]:
# Renaming some columns for part4
morticd10_part4_df= morticd10_part4_df.rename(columns = {'Country' : 'Country_code',
                                                      'Deaths1' : "Total_deaths", 'IM_Deaths1' : 'Mort_day0',
                                                      'IM_Deaths2' : 'Mort_day1_6' , 
                                                      'IM_Deaths3' : 'Mort_day7_27',
                                                      'IM_Deaths4': 'Mort_day28_365'})
morticd10_part4_df

,Country_code,Year,List,Cause,Sex,Total_deaths,Mort_day0,Mort_day1_6,Mort_day7_27,Mort_day28_365,mort_1_4,mort_5_24,mort_25_44,mort_45_64,mort_65_84,mort_85_95+,Deaths26
0,1400,2013,101,1000,1,383,5.0,1.0,2.0,4.0,0.0,6.0,44.0,103.0,218.0,0.0,0.0
1,1400,2013,101,1000,2,334,2.0,7.0,3.0,5.0,2.0,3.0,20.0,60.0,232.0,0.0,0.0
2,1400,2013,101,1001,1,28,0.0,0.0,0.0,0.0,0.0,0.0,6.0,11.0,11.0,0.0,0.0
3,1400,2013,101,1001,2,30,0.0,2.0,2.0,1.0,0.0,0.0,0.0,6.0,19.0,0.0,0.0
4,1400,2013,101,1002,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880537,5020,2016,104,Y870,2,2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
880538,5020,2016,104,Y883,1,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,1.0,0.0
880539,5020,2016,104,Y883,2,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,3.0,0.0
880540,5020,2016,104,Y899,1,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,0.0


In [12]:
part4_cols = morticd10_part4_df.columns
part4_cols

Index(['Country_code', 'Year', 'List', 'Cause', 'Sex', 'Total_deaths',
       'Mort_day0', 'Mort_day1_6', 'Mort_day7_27', 'Mort_day28_365',
       'mort_1_4', 'mort_5_24', 'mort_25_44', 'mort_45_64', 'mort_65_84',
       'mort_85_95+', 'Deaths26'],
      dtype='object')

### Concatenate part4 and part5.Keepking only the Total_deaths column together with Country_code, Year, list, Cause, Sex 


In [13]:
part4_cols = morticd10_part4_df.columns
part4_cols

Index(['Country_code', 'Year', 'List', 'Cause', 'Sex', 'Total_deaths',
       'Mort_day0', 'Mort_day1_6', 'Mort_day7_27', 'Mort_day28_365',
       'mort_1_4', 'mort_5_24', 'mort_25_44', 'mort_45_64', 'mort_65_84',
       'mort_85_95+', 'Deaths26'],
      dtype='object')

In [14]:
#Keeping only desired columns to study the total deaths per cause
morticd10_part4_totaldeath_df = morticd10_part4_df[['Country_code', 'Year', 'Cause', 'Sex', 'Total_deaths']]
morticd10_part4_totaldeath_df

,Country_code,Year,Cause,Sex,Total_deaths
0,1400,2013,1000,1,383
1,1400,2013,1000,2,334
2,1400,2013,1001,1,28
3,1400,2013,1001,2,30
4,1400,2013,1002,1,0
...,...,...,...,...,...
880537,5020,2016,Y870,2,2
880538,5020,2016,Y883,1,6
880539,5020,2016,Y883,2,7
880540,5020,2016,Y899,1,6


In [15]:
#Keeping only desired columns to study the total deaths per cause
morticd10_part5_totaldeath_df = morticd10_part5_df[['Country_code', 'Year', 'Cause', 'Sex', 'Total_deaths']]
morticd10_part5_totaldeath_df

,Country_code,Year,Cause,Sex,Total_deaths
0,4303,2017,1000,1,281784
1,4303,2017,1000,2,292339
2,4303,2017,1001,1,6198
3,4303,2017,1001,2,2516
4,4303,2017,1002,1,0
...,...,...,...,...,...
1023255,5020,2022,Y871,1,1
1023256,5020,2022,Y883,1,4
1023257,5020,2022,Y883,2,3
1023258,5020,2022,Y890,2,1


### Concatenate the two DataFrames (with total death per cause) part4 and part5 

In [16]:
# Concatenate the two DataFrames
morticd10_totaldeath_per_cause_2013_2022_df = pd.concat([morticd10_part4_totaldeath_df, morticd10_part5_totaldeath_df], ignore_index=True)
morticd10_totaldeath_per_cause_2013_2022_df

,Country_code,Year,Cause,Sex,Total_deaths
0,1400,2013,1000,1,383
1,1400,2013,1000,2,334
2,1400,2013,1001,1,28
3,1400,2013,1001,2,30
4,1400,2013,1002,1,0
...,...,...,...,...,...
1903797,5020,2022,Y871,1,1
1903798,5020,2022,Y883,1,4
1903799,5020,2022,Y883,2,3
1903800,5020,2022,Y890,2,1


## Section to determine the most deadly causes.

## Get the total death per cause group. Exclude the causes 'AAA' and '1000' as they represent total death for all causes combined

In [17]:
# Exclude the causes 'AAA' and '1000' as they represent total death for all causes combined
excluded_causes = ['AAA', '1000']
filtered_df = morticd10_totaldeath_per_cause_2013_2022_df[~morticd10_totaldeath_per_cause_2013_2022_df['Cause'].isin(excluded_causes)]

# Extract the first character of each cause
filtered_df['Cause_Group'] = filtered_df['Cause'].str[0]

# Aggregate total deaths by cause group across all years
total_deaths_by_cause_group = filtered_df.groupby('Cause_Group')['Total_deaths'].sum().reset_index()

# Rename columns for clarity
total_deaths_by_cause_group.columns = ['Cause_Group', 'Total_Deaths']

# Sort by total deaths in descending order (optional)
total_deaths_by_cause_group_2013_2022 = total_deaths_by_cause_group.sort_values(by='Total_Deaths', ascending=False)

# Display the result
print(total_deaths_by_cause_group_2013_2022)

   Cause_Group  Total_Deaths
9            I      46731302
3            C      31574262
0            1      31404956
10           J      15129220
18           R       7715749
11           K       6910410
5            E       6817223
7            G       6246297
22           X       4975907
6            F       4582439
14           N       3966262
19           U       3523912
1            A       3095119
21           W       2452601
20           V       2030845
4            D       1709658
23           Y       1204836
2            B       1190501
16           P       1127976
13           M        769881
17           Q        768683
12           L        402453
15           O         85276
8            H         11383


C:\Users\Owner\AppData\Local\Temp\ipykernel_34092\677426478.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Cause_Group'] = filtered_df['Cause'].str[0]


# Studying the country code and adding geocoordinates

In [18]:
country_code_df

,ï»¿country,name
0,1010,Algeria
1,1020,Angola
2,1025,Benin
3,1030,Botswana
4,1035,Burkina Faso
...,...,...
222,5197,Samoa
223,5198,Solomon Islands
224,5200,Tonga
225,5205,Tuvalu


In [19]:
# Renaming the country column to Country_code
country_code_df= country_code_df.rename(columns = {'ï»¿country' : 'Country_code',
                                                   'name' : 'Country_name'})
country_code_df

,Country_code,Country_name
0,1010,Algeria
1,1020,Angola
2,1025,Benin
3,1030,Botswana
4,1035,Burkina Faso
...,...,...
222,5197,Samoa
223,5198,Solomon Islands
224,5200,Tonga
225,5205,Tuvalu


## Converting the numeric country code to ISO country codes

In [20]:
# Function to fetch ISO Alpha-3 codes using pycountry
def get_iso_alpha3(country_name):
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except LookupError:
        return None  # Return None if the country name is not found

# Add ISO_Alpha3 column
country_code_df['ISO_Alpha3'] = country_code_df['Country_name'].apply(get_iso_alpha3)

# Display the updated DataFrame
country_code_df.head()

,Country_code,Country_name,ISO_Alpha3
0,1010,Algeria,DZA
1,1020,Angola,AGO
2,1025,Benin,BEN
3,1030,Botswana,BWA
4,1035,Burkina Faso,BFA


In [21]:
morticd10_totaldeath_per_cause_2013_2022_df.head()

,Country_code,Year,Cause,Sex,Total_deaths
0,1400,2013,1000,1,383
1,1400,2013,1000,2,334
2,1400,2013,1001,1,28
3,1400,2013,1001,2,30
4,1400,2013,1002,1,0


## Merge the mortality data with country dataframe

In [22]:
# Merge the mortality dataframe with the country data
# Match by 'Country_code' from the CSV and 'Country_code' from the world dataset
country_mortality_2013_2022_df = pd.merge(
    morticd10_totaldeath_per_cause_2013_2022_df,
    country_code_df,
    on='Country_code',
    how='left'  # Use 'left' join to retain all rows from the mortality data
)
country_mortality_2013_2022_df

,Country_code,Year,Cause,Sex,Total_deaths,Country_name,ISO_Alpha3
0,1400,2013,1000,1,383,Seychelles,SYC
1,1400,2013,1000,2,334,Seychelles,SYC
2,1400,2013,1001,1,28,Seychelles,SYC
3,1400,2013,1001,2,30,Seychelles,SYC
4,1400,2013,1002,1,0,Seychelles,SYC
...,...,...,...,...,...,...,...
1903797,5020,2022,Y871,1,1,Australia,AUS
1903798,5020,2022,Y883,1,4,Australia,AUS
1903799,5020,2022,Y883,2,3,Australia,AUS
1903800,5020,2022,Y890,2,1,Australia,AUS


In [23]:
# Drop rows containing NaN values
cleaned_country_mortality_2013_2022_df= country_mortality_2013_2022_df.dropna()
cleaned_country_mortality_2013_2022_df

,Country_code,Year,Cause,Sex,Total_deaths,Country_name,ISO_Alpha3
0,1400,2013,1000,1,383,Seychelles,SYC
1,1400,2013,1000,2,334,Seychelles,SYC
2,1400,2013,1001,1,28,Seychelles,SYC
3,1400,2013,1001,2,30,Seychelles,SYC
4,1400,2013,1002,1,0,Seychelles,SYC
...,...,...,...,...,...,...,...
1903797,5020,2022,Y871,1,1,Australia,AUS
1903798,5020,2022,Y883,1,4,Australia,AUS
1903799,5020,2022,Y883,2,3,Australia,AUS
1903800,5020,2022,Y890,2,1,Australia,AUS


In [27]:
# Add a new column for cause group (first letter of Cause)
# Add a new column for cause group (first letter of Cause) using .loc
cleaned_country_mortality_2013_2022_df = cleaned_country_mortality_2013_2022_df.copy()  # Ensure it's a copy, not a view
cleaned_country_mortality_2013_2022_df.loc[:, 'Cause_Group'] = cleaned_country_mortality_2013_2022_df['Cause'].str[0]

# Aggregate Total_deaths by Cause_Group
aggregated_mortality_byCauseGroup_2013_2022_df = cleaned_country_mortality_2013_2022_df.groupby(['Country_code','Country_name','ISO_Alpha3', 'Year','Cause_Group','Sex'])['Total_deaths'].sum().reset_index()

# Display the result
aggregated_mortality_byCauseGroup_2013_2022_df

,Country_code,Country_name,ISO_Alpha3,Year,Cause_Group,Sex,Total_deaths
0,1125,Egypt,EGY,2013,A,1,288915
1,1125,Egypt,EGY,2013,A,2,231400
2,1125,Egypt,EGY,2013,B,1,4905
3,1125,Egypt,EGY,2013,B,2,2972
4,1125,Egypt,EGY,2013,C,1,18119
...,...,...,...,...,...,...,...
34153,5198,Solomon Islands,SLB,2018,W,1,4
34154,5198,Solomon Islands,SLB,2018,W,2,1
34155,5198,Solomon Islands,SLB,2018,X,1,2
34156,5198,Solomon Islands,SLB,2018,Y,1,3


# Save the cleaned data from 2013-2022 and the aggregated total death by Cause_Group with country Alpha3 codes

In [32]:
# Export file as a CSV, without the Pandas index, but with the header
cleaned_country_mortality_2013_2022_df.to_csv("cleaneddata/cleaned_country_mortality_2013_2022_df.csv", index=False, header=True)
aggregated_mortality_byCauseGroup_2013_2022_df.to_csv("cleaneddata/aggregated_mortality_byCauseGroup_2013_2022_df.csv", index=False, header=True)

## Function to visualize deaths for a selected cause

In [29]:
# Function to visualize deaths for a selected cause
def visualize_cause_deaths(cause):
    # Filter data for the selected cause
    cause_data = cleaned_country_mortality_2013_2022_df[cleaned_country_mortality_2013_2022_df['Cause'] == cause]
    
    # Aggregate total deaths by country for the selected cause
    country_death_data = cause_data.groupby('ISO_Alpha3')['Total_deaths'].sum().reset_index()
    
    # Rename columns for clarity
    country_death_data.columns = ['ISO_Alpha3', 'Total_Deaths']
    
    # Plot the data on a map
    fig = px.scatter_geo(
        country_death_data,
        locations='ISO_Alpha3',  # Country codes (ISO Alpha-3 recommended)
        size='Total_Deaths',
        title=f'Total Deaths for Cause: {cause}',
        projection='natural earth',
        size_max=50,
        hover_name='ISO_Alpha3',
    )
    fig.update_geos(showcountries=True, showframe=True)
    fig.show()

# List of unique causes for dropdown menu
unique_causes = cleaned_country_mortality_2013_2022_df['Cause'].unique()

# Create an interactive dropdown to select a cause
interact(visualize_cause_deaths, cause=unique_causes)

interactive(children=(Dropdown(description='cause', options=('1000', '1001', '1002', '1003', '1004', '1005', '…

<function __main__.visualize_cause_deaths(cause)>

## Function to visualize deaths for a selected cause and year

In [30]:
# Function to visualize deaths for a selected cause and year
def visualize_cause_deaths(cause, year):
    # Filter data for the selected cause and year
    filtered_data = cleaned_country_mortality_2013_2022_df[
        (cleaned_country_mortality_2013_2022_df['Cause'] == cause) & 
        (cleaned_country_mortality_2013_2022_df['Year'] == year)
    ]
    
    # Aggregate total deaths by country for the selected cause and year
    country_death_data = filtered_data.groupby('ISO_Alpha3')['Total_deaths'].sum().reset_index()
    
    # Rename columns for clarity
    country_death_data.columns = ['ISO_Alpha3', 'Total_Deaths']
    
    # Plot the data on a map
    fig = px.scatter_geo(
        country_death_data,
        locations='ISO_Alpha3',  # Country codes (ISO Alpha-3 recommended)
        size='Total_Deaths',
        title=f'Total Deaths for Cause: {cause} in Year: {year}',
        projection='natural earth',
        size_max=50,
        hover_name='ISO_Alpha3',
    )
    fig.update_geos(showcountries=True, showframe=True)
    fig.show()

# List of unique causes and years for dropdown menus
unique_causes = cleaned_country_mortality_2013_2022_df['Cause'].unique()
unique_years = cleaned_country_mortality_2013_2022_df['Year'].dropna().unique()

# Create an interactive dropdown to select a cause and a year
interact(visualize_cause_deaths, cause=unique_causes, year=unique_years)

interactive(children=(Dropdown(description='cause', options=('1000', '1001', '1002', '1003', '1004', '1005', '…

<function __main__.visualize_cause_deaths(cause, year)>

## Adding code to make bubbles of each country to have different colors

In [31]:
# Function to visualize deaths for a selected cause and year
def visualize_cause_deaths(cause, year):
    # Filter data for the selected cause and year
    filtered_data = cleaned_country_mortality_2013_2022_df[
        (cleaned_country_mortality_2013_2022_df['Cause'] == cause) & 
        (cleaned_country_mortality_2013_2022_df['Year'] == year)
    ]
    
    # Check if there is data to display
    if filtered_data.empty:
        print(f"No data available for Cause: {cause} in Year: {year}")
        return
    
    # Aggregate total deaths by country for the selected cause and year
    country_death_data = filtered_data.groupby('ISO_Alpha3')['Total_deaths'].sum().reset_index()
    country_death_data.columns = ['ISO_Alpha3', 'Total_Deaths']
    
    # Assign unique colors to each country
    country_death_data['Country_Color'] = country_death_data['ISO_Alpha3']
    
    # Plot the data on a map
    fig = px.scatter_geo(
        country_death_data,
        locations='ISO_Alpha3', 
        size='Total_Deaths',
        color='Country_Color',  # Different color for each country
        title=f'Total Deaths for Cause: {cause} in Year: {year}',
        projection='natural earth',
        size_max=50,
        hover_name='ISO_Alpha3',
    )
    fig.update_geos(showcountries=True, showframe=True)
    fig.show()

# List of unique causes and years for dropdown menus
unique_causes = sorted(cleaned_country_mortality_2013_2022_df['Cause'].unique())
unique_years = sorted(cleaned_country_mortality_2013_2022_df['Year'].dropna().unique())

# Create an interactive dropdown to select a cause and a year
interact(visualize_cause_deaths, cause=unique_causes, year=unique_years)

interactive(children=(Dropdown(description='cause', options=('1000', '1001', '1002', '1003', '1004', '1005', '…

<function __main__.visualize_cause_deaths(cause, year)>

## Adding a slider horizontal bar to choose the year

In [77]:
# Function to visualize deaths for a selected cause and year
def visualize_cause_deaths(cause, year):
    # Filter data for the selected cause and year
    filtered_data = cleaned_country_mortality_2013_2022_df[
        (cleaned_country_mortality_2013_2022_df['Cause'] == cause) & 
        (cleaned_country_mortality_2013_2022_df['Year'] == year)
    ]
    
    # Check if there is data to display
    if filtered_data.empty:
        print(f"No data available for Cause: {cause} in Year: {year}")
        return
    
    # Aggregate total deaths by country for the selected cause and year
    country_death_data = filtered_data.groupby('ISO_Alpha3')['Total_deaths'].sum().reset_index()
    country_death_data.columns = ['ISO_Alpha3', 'Total_Deaths']
    
    # Plot the data on a map
    fig = px.scatter_geo(
        country_death_data,
        locations='ISO_Alpha3', 
        size='Total_Deaths',
        color='ISO_Alpha3',  # Different color for each country
        title=f'Total Deaths for Cause: {cause} in Year: {year}',
        projection='natural earth',
        size_max=50,
        hover_name='ISO_Alpha3',
    )
    fig.update_geos(showcountries=True, showframe=True)
    fig.show()

# List of unique causes and years for dropdown and slider
unique_causes = sorted(cleaned_country_mortality_2013_2022_df['Cause'].unique())
min_year = int(cleaned_country_mortality_2013_2022_df['Year'].min())
max_year = int(cleaned_country_mortality_2013_2022_df['Year'].max())

# Create an interactive dropdown for cause and slider for year
interact(
    visualize_cause_deaths, 
    cause=unique_causes, 
    year=IntSlider(min=min_year, max=max_year, step=1, value=min_year, description='Year')
)

interactive(children=(Dropdown(description='cause', options=('1000', '1001', '1002', '1003', '1004', '1005', '…

<function __main__.visualize_cause_deaths(cause, year)>

## Making visualizations based on cause_group

In [78]:
# Function to visualize deaths for a grouped cause and year
def visualize_grouped_cause_deaths(cause_group, year):
    # Add a new column for the first character of the cause
    cleaned_country_mortality_2013_2022_df['Cause_Group'] = cleaned_country_mortality_2013_2022_df['Cause'].str[0]
    
    # Filter data for the selected cause group and year
    filtered_data = cleaned_country_mortality_2013_2022_df[
        (cleaned_country_mortality_2013_2022_df['Cause_Group'] == cause_group) & 
        (cleaned_country_mortality_2013_2022_df['Year'] == year)
    ]
    
    # Check if there is data to display
    if filtered_data.empty:
        print(f"No data available for Cause Group: {cause_group} in Year: {year}")
        return
    
    # Aggregate total deaths by country for the selected cause group and year
    country_death_data = filtered_data.groupby('ISO_Alpha3')['Total_deaths'].sum().reset_index()
    country_death_data.columns = ['ISO_Alpha3', 'Total_Deaths']
    
    # Plot the data on a map
    fig = px.scatter_geo(
        country_death_data,
        locations='ISO_Alpha3', 
        size='Total_Deaths',
        color='ISO_Alpha3',  # Different color for each country
        title=f'Total Deaths for Cause Group: {cause_group} in Year: {year}',
        projection='natural earth',
        size_max=50,
        hover_name='ISO_Alpha3',
    )
    fig.update_geos(showcountries=True, showframe=True)
    fig.show()

# Create a list of unique cause groups and years for dropdown and slider
unique_cause_groups = sorted(cleaned_country_mortality_2013_2022_df['Cause'].str[0].unique())
min_year = int(cleaned_country_mortality_2013_2022_df['Year'].min())
max_year = int(cleaned_country_mortality_2013_2022_df['Year'].max())

# Create an interactive dropdown for cause group and slider for year
interact(
    visualize_grouped_cause_deaths, 
    cause_group=unique_cause_groups, 
    year=IntSlider(min=min_year, max=max_year, step=1, value=min_year, description='Year')
)

interactive(children=(Dropdown(description='cause_group', options=('1', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'…

<function __main__.visualize_grouped_cause_deaths(cause_group, year)>